# The Pokemon Cookbook
This cookbook teaches you the concepts of the InfluxDB 3.0 Python Client library using a novel example of Pokemon data. The scenerio is to keep track of each trainer and the number of differnt pokemon they have caught.
<p align="center">
  <img height="300" src="https://www.nicepng.com/png/full/62-622961_no-one-knows-if-people-eat-pokmon-png.png">
</p>

In [1]:

# Here we include all the imports required from influxdb_client_3 
from influxdb_client_3 import InfluxDBClient3, InfluxDBError, WriteOptions, write_client_options
import pandas as pd
import random

## Writing Data
The first step is to write data into InfluxDB. We will use the `write_api` to write data into InfluxDB. In this example we are going to utilise `batching mode` to write data in batches. This is the most efficient way to write data into InfluxDB. To do this we are going to first setup some paramters for our client.

In [2]:
# This class handles the callbacks for the batching
class BatchingCallback(object):

    def success(self, conf, data: str):
        print(f"Written batch: {conf}")

    def error(self, conf, data: str, exception: InfluxDBError):
        print(f"Cannot write batch: {conf}, data: {data} due: {exception}")

    def retry(self, conf, data: str, exception: InfluxDBError):
        print(f"Retryable error occurs for batch: {conf}, data: {data} retry: {exception}")

callback = BatchingCallback()

# This is the configuration for the batching. This is wrapped in a WriteOptions object. Within this example you
# can see the different options that can be set for the batching.
# Batch size is the number of points to write before the batch is written to the server.
# Flush interval is the time in milliseconds to wait before the batch is written to the server.
# Jitter interval is the time in milliseconds to wait before the batch is written to the server.
# Retry interval is the time in milliseconds to wait before retrying a failed batch.
# Max retries is the maximum number of times to retry a failed batch.
# exponential base is the base for the exponential retry delay.
write_options = WriteOptions(batch_size=100,
                                        flush_interval=10_000,
                                        jitter_interval=2_000,
                                        retry_interval=5_000,
                                        max_retries=5,
                                        max_retry_delay=30_000,
                                        exponential_base=2)


# This is the configuration for the write client. This is wrapped in a WriteClientOptions object.
# As you can see we incldue the BatchingCallback object we created earlier, plus the write_options.
wco = write_client_options(success_callback=callback.success,
                          error_callback=callback.error,
                          retry_callback=callback.retry,
                          WriteOptions=write_options 
                        )

## Client Setup
Now that we have done the inital configurations of our write paramters its time to include these within our client initalization. The InfluxDB 3.0 Client can both write and query data. For now we will use it to write data based upon our configuration.

In [3]:
# In this example we are using the InfluxDBClient3 object to connect to the InfluxDB Cloud instance.
# We are also passing in the write_client_options we created earlier.
# The token, host and org are all required to connect to the InfluxDB Serverless instance.
# Note: that Org is optional with Dedicated instances.
client = InfluxDBClient3(
    token="",
    host="eu-central-1-1.aws.cloud2.influxdata.com",
    org="6a841c0c08328fb1",
    database="pokemon-codex", enable_gzip=True, write_client_options=wco)

now =  pd.Timestamp.now(tz='UTC').floor('ms')

# Lists of possible trainers
trainers = ["ash", "brock", "misty", "gary", "jessie", "james"]

# Read the CSV into a DataFrame. (Credit to @ritchie46 for the dataset)
pokemon_df = pd.read_csv("./pokemon.csv")

# Creating an empty list to store the data
data = []

# Dictionary to keep track of the number of times each trainer has caught each Pokémon
trainer_pokemon_counts = {}

# Number of entries we want to create
num_entries = 1000

### Generate some data
Now that we have our client setup lets start generating some data we can write to InfluxDB. Following the Pokemon example we will create a list of trainers and the number of pokemon they have caught. Trainers will catch pokemon randomly selected from our list stored within the Pandas DataFrame `pokemon_df`.

In [4]:
from IPython.display import display, HTML

# Generating random data
for i in range(num_entries):
    trainer = random.choice(trainers)
    
    # Randomly select a row from pokemon_df
    random_pokemon = pokemon_df.sample().iloc[0]
    caught = random_pokemon['Name']
    
    # Count the number of times this trainer has caught this Pokémon
    if (trainer, caught) in trainer_pokemon_counts:
        trainer_pokemon_counts[(trainer, caught)] += 1
    else:
        trainer_pokemon_counts[(trainer, caught)] = 1
    
    # Get the number for this combination of trainer and Pokémon
    num = trainer_pokemon_counts[(trainer, caught)]

    entry = {
        "trainer": trainer,
        "id": f"{0000 + random_pokemon['#']:04d}",
        "num": str(num),
        "name": caught,
        "level": random.randint(5, 20),
        "attack": random_pokemon['Attack'],
        "defense": random_pokemon['Defense'],
        "hp": random_pokemon['HP'],
        "speed": random_pokemon['Speed'],
        "type1": random_pokemon['Type 1'],
        "type2": random_pokemon['Type 2'],
        "timestamp": now
    }
    data.append(entry)

# Convert the list of dictionaries to a DataFrame
caught_pokemon_df = pd.DataFrame(data).set_index('timestamp')

# Print the DataFrame
display(caught_pokemon_df)

,trainer,id,num,name,level,attack,defense,hp,speed,type1,type2
timestamp,,,,,,,,,,,
2023-07-24 15:32:24.872000+00:00,ash,0149,1,Dragonite,20,134,95,91,80,Dragon,Flying
2023-07-24 15:32:24.872000+00:00,gary,0084,1,Doduo,11,85,45,35,75,Normal,Flying
2023-07-24 15:32:24.872000+00:00,jessie,0095,1,Onix,14,45,160,35,70,Rock,Ground
2023-07-24 15:32:24.872000+00:00,brock,0024,1,Arbok,14,85,69,60,80,Poison,NaN
2023-07-24 15:32:24.872000+00:00,gary,0035,1,Clefairy,15,45,48,70,35,Fairy,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24 15:32:24.872000+00:00,gary,0146,2,Moltres,6,100,90,90,90,Fire,Flying
2023-07-24 15:32:24.872000+00:00,misty,0123,3,Scyther,18,110,80,70,105,Bug,Flying
2023-07-24 15:32:24.872000+00:00,brock,0007,2,Squirtle,17,48,65,44,43,Water,NaN


### Write Data to InfluxDB
We will now write our newley created trainer data to InfluxDB. To do this we simply call client.write and pass in our dataframe. We then provide a static measurement name of `caught`. We also provide a series of tags to help identify our data. In this case we use the columns `['trainer', 'id', 'num']`. Note that we didn't provide our time column this is due to the fact we set this column as our `dataframe_index`. This means that the index column will be used as the time column.

In [5]:
import time

try:
    client.write(caught_pokemon_df, data_frame_measurement_name='caught',
             data_frame_tag_columns=['trainer', 'id', 'num'])
except Exception as e:
    print(f"Error writing point: {e}")

# Wait for the batch to be written
time.sleep(2)
    

Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')


## Querying Data
We have now stored 1000 registered pokemon catches within InfluxDB. We can now query this data using the InfluxDB 3.0 Python Client to gain some insights into our data. We are going to use Plotly to visualise our data.

In [6]:
# These are just some library imports for Plotly so we can make use of the interactive graphs.
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"

# Lets start with a simple query to understand our schema.
query = '''SHOW COLUMNS FROM caught'''

# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.
table = client.query(query=query, language='sql', mode='pandas')

display(table)

,table_catalog,table_schema,table_name,column_name,data_type,is_nullable
0,public,iox,caught,attack,Int64,YES
1,public,iox,caught,defense,Int64,YES
2,public,iox,caught,hp,Int64,YES
3,public,iox,caught,id,"Dictionary(Int32, Utf8)",YES
4,public,iox,caught,level,Int64,YES
5,public,iox,caught,name,Utf8,YES
6,public,iox,caught,num,"Dictionary(Int32, Utf8)",YES
7,public,iox,caught,speed,Int64,YES
8,public,iox,caught,time,"Timestamp(Nanosecond, None)",NO
9,public,iox,caught,trainer,"Dictionary(Int32, Utf8)",YES


### Simple InfluxQL Query
The first query we will run is a simple InfluxQL query to get the number of pokemon caught by each trainer. We will then use Plotly to visualise this data.

In [7]:
# Lets start with a simple query to understand our schema.
query = '''SELECT count("name") FROM caught WHERE time > now() - 1h GROUP BY trainer'''

# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.
table = client.query(query=query, language='influxql', mode='pandas')

fig1 = px.bar(table, x="trainer", y="count",color='trainer' ,title='Number of Pokémon caught in the last hour')
fig1.show()

In [8]:
# Lets start with a simple query to understand our schema.
query = '''SELECT count("name") FROM caught WHERE time > now() - 1h GROUP BY trainer,type1'''

# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.
table = client.query(query=query, language='influxql' , mode='pandas')

fig2 = px.bar(table, x="trainer", y="count", color='type1', barmode= 'group', title='Number of Pokémon caught in the last hour grouped by type')
fig2.show()

### Working with Arrow tables
So within the last section we discussed converting returned queries directly to Pandas Dataframes. However, we can also utilise their raw format, Arrow tables. Arrow tables are a columnar format that is more efficient for working with data.

Lets first start by adding more data with a random timestamp between now and 1 hour ago

In [9]:
num_entries = 10000 # You can reduce this if required. The more data the more interesting the results.
now =  pd.Timestamp.now(tz='UTC').floor('ms')
data = []


# Generating random data
for i in range(num_entries):
    # Randomise the timestamp
    timestamp = now - pd.Timedelta(minutes=random.randint(0, 60))
    trainer = random.choice(trainers)
    
    # Randomly select a row from pokemon_df
    random_pokemon = pokemon_df.sample().iloc[0]
    caught = random_pokemon['Name']
    
    # Count the number of times this trainer has caught this Pokémon
    if (trainer, caught) in trainer_pokemon_counts:
        trainer_pokemon_counts[(trainer, caught)] += 1
    else:
        trainer_pokemon_counts[(trainer, caught)] = 1
    
    # Get the number for this combination of trainer and Pokémon
    num = trainer_pokemon_counts[(trainer, caught)]

    entry = {
        "trainer": trainer,
        "id": f"{0000 + random_pokemon['#']:04d}",
        "num": str(num),
        "name": caught,
        "level": random.randint(5, 20),
        "attack": random_pokemon['Attack'],
        "defense": random_pokemon['Defense'],
        "hp": random_pokemon['HP'],
        "speed": random_pokemon['Speed'],
        "type1": random_pokemon['Type 1'],
        "type2": random_pokemon['Type 2'],
        "timestamp": timestamp
    }
    data.append(entry)

# Convert the list of dictionaries to a DataFrame
caught_pokemon_df = pd.DataFrame(data).set_index('timestamp')

# Print the DataFrame
display(caught_pokemon_df)

try:
    client.write(caught_pokemon_df, data_frame_measurement_name='kanto',
             data_frame_tag_columns=['trainer', 'id', 'num'])
except Exception as e:
    print(f"Error writing point: {e}")

# Wait for the batch to be written
time.sleep(5)


,trainer,id,num,name,level,attack,defense,hp,speed,type1,type2
timestamp,,,,,,,,,,,
2023-07-24 15:15:28.882000+00:00,james,0077,3,Ponyta,12,85,55,50,90,Fire,NaN
2023-07-24 14:53:28.882000+00:00,brock,0099,1,Kingler,15,130,115,55,75,Water,NaN
2023-07-24 15:31:28.882000+00:00,misty,0051,1,Dugtrio,8,80,50,35,120,Ground,NaN
2023-07-24 15:16:28.882000+00:00,gary,0102,1,Exeggcute,13,40,80,60,40,Grass,Psychic
2023-07-24 15:00:28.882000+00:00,misty,0007,1,Squirtle,20,48,65,44,43,Water,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24 14:55:28.882000+00:00,ash,0118,13,Goldeen,6,67,60,45,63,Water,NaN
2023-07-24 15:29:28.882000+00:00,misty,0098,18,Krabby,5,105,90,30,50,Water,NaN
2023-07-24 14:39:28.882000+00:00,james,0040,14,Wigglytuff,18,70,45,140,45,Normal,Fairy


Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')


Now lets return the result as an Arrow table. We can recreate he same aggregation we did on server side with the pyarrow library. the `mode='all'` parameter tells InfluxDB to return all data within the query result as a Arrow table. We can then use the `to_pandas()` method to convert the Arrow table to a Pandas DataFrame.

In [10]:
table = client.query(query='''SELECT * FROM kanto ORDER BY time''', language='influxql', mode='all')
print(table)

# PyArrow Aggregation
aggregation = table.group_by(["trainer", "type1"]).aggregate([("name", "count")]).to_pandas()
fig3 = px.bar(aggregation, x="trainer", y="name_count", color='type1', barmode= 'group', title='Number of Pokémon caught in the last hour grouped by type')
fig3.show()

pyarrow.Table
iox::measurement: string not null
time: timestamp[ns] not null
attack: int64
defense: int64
hp: int64
id: string
level: int64
name: string
num: string
speed: int64
trainer: string
type1: string
type2: string
----
iox::measurement: [["kanto","kanto","kanto","kanto","kanto",...,"kanto","kanto","kanto","kanto","kanto"],["kanto","kanto","kanto","kanto","kanto",...,"kanto","kanto","kanto","kanto","kanto"],...,["kanto","kanto","kanto","kanto","kanto",...,"kanto","kanto","kanto","kanto","kanto"],["kanto","kanto","kanto","kanto","kanto",...,"kanto","kanto","kanto","kanto","kanto"]]
time: [[2023-07-24 13:30:08.047000000,2023-07-24 13:30:08.047000000,2023-07-24 13:30:08.047000000,2023-07-24 13:30:08.047000000,2023-07-24 13:30:08.047000000,...,2023-07-24 14:12:34.198000000,2023-07-24 14:12:34.198000000,2023-07-24 14:12:34.198000000,2023-07-24 14:12:34.198000000,2023-07-24 14:12:34.198000000],[2023-07-24 14:12:34.198000000,2023-07-24 14:12:34.198000000,2023-07-24 14:12:34.198000000,2

### Saving to file
We can also save our query results to file. This is useful if we want to save our data for later use. We can save our data in a number of formats including CSV, JSON, Parquet and Apache Arrow. Lets save our data as a parquet file.

In [11]:
import pyarrow.parquet as pq

# Write the table to a parquet file
pq.write_table(table, 'kanto.parquet')

### Uploading a file to InfluxDB
This allows to show show off another feature of the InfluxDB 3.0 Python Client. We can parse our file directly to InfluxDB. This is useful if we want to upload data from a local file. Lets upload our parquet file to InfluxDB.

In [12]:
client.write_file('kanto.parquet', measurement_name='kanto2', database='pokemon-codex', timestamp_column='time', tag_columns=['trainer', 'id', 'num'])
time.sleep(2)

Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns

### Group by time query
Finally lets run a group by time query to get the number of pokemon caught by each trainer over the last hour grouped into 10 minute intervals. We will then use Plotly to visualise this data.

In [13]:
query='''SELECT count("name") FROM kanto2 WHERE time > now() - 1h GROUP BY time(10m),trainer'''
table = client.query(query=query, language='influxql', mode='all')

fig4 = px.line(table, x="time", y="count", color='trainer', title='Number of Pokémon caught in the last hour grouped by trainer and time')
fig4.show()

Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns')
Written batch: ('pokemon-codex', '6a841c0c08328fb1', 'ns

# Conclusion
We have now covered the basics of the InfluxDB 3.0 Python Client. I hope you found this novel cook book informative and fun. If you have any questions, bugs or feature requests please raise an issue on the [GitHub repo](https://github.com/InfluxCommunity/influxdb3-python/issues)

<p align="center">
  <img height="150" src="https://i.pinimg.com/originals/18/15/44/181544facabe62d30c52e94b369f0f3a.png">
</p>
